In [1]:
!gdown --id 11Svg8Vze81-r-43w1eo3gOBs4oCUfIny
!gdown --id 1f3W5Y_fh2au6Kf_tPawYihxzLdFqzHGy
!gdown --id 1f6PUWUtqpdGz25VPuJtoGhAgVH12qted #download th-bpe files
!unzip bert_th.zip
!unzip th_wiki_bpe.zip
!git clone https://github.com/thaikeras/bert
!mv bert/* ./

Downloading...
From: https://drive.google.com/uc?id=11Svg8Vze81-r-43w1eo3gOBs4oCUfIny
To: /content/AACL_BERT_TH.hdf5
421MB [00:07, 57.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f3W5Y_fh2au6Kf_tPawYihxzLdFqzHGy
To: /content/bert_th.zip
857MB [00:15, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f6PUWUtqpdGz25VPuJtoGhAgVH12qted
To: /content/th_wiki_bpe.zip
100% 560k/560k [00:00<00:00, 37.4MB/s]
Archive:  bert_th.zip
   creating: bert_base_th/
   creating: bert_base_th/.ipynb_checkpoints/
  inflating: bert_base_th/config.json  
  inflating: bert_base_th/pytorch_model.bin  
  inflating: bert_base_th/tf_model.h5  
Archive:  th_wiki_bpe.zip
  inflating: th.wiki.bpe.op25000.model  
  inflating: th.wiki.bpe.op25000.vocab  
Cloning into 'bert'...
remote: Enumerating objects: 275, done.
remote: Total 275 (delta 0), reused 0 (delta 0), pack-reused 275
Receiving objects: 100% (275/275), 201.42 KiB | 292.00 KiB/s, done.
Resolving deltas: 100% (151/151), done.


In [3]:
!pip install transformers
!pip install pythainlp

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 14.1MB/s 
     |████████████████████████████████| 1.1MB 16.8MB/s 
     |████████████████████████████████| 3.0MB 23.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8f8479609bb448c162b30bf5b748a4a10559a2f55ea3bde5bd696bf89da18cb1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 13.1MB 242kB/s 
     |████████████████████████████████| 747kB 41.3MB/s 


In [4]:
from transformers import TFBertModel, TFBertForTokenClassification, BertConfig
import tensorflow as tf
import transformers
import re
import json
import tokenization as tok
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tqdm import tqdm_notebook
from string import punctuation
from pythainlp import thai_digits
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
from tensorflow.keras.utils import to_categorical

config = BertConfig.from_pretrained('bert_base_th')
tf.gfile = tf.io.gfile

In [7]:
from pythainlp import word_tokenize

In [6]:
MAX_SEQUENCE_LENGTH = 110
EPOCHS = 30

In [52]:
text = [word_tokenize("สวัสดีครับ")]
text

[['สวัสดี', 'ครับ']]

In [46]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        return input_ids, input_mask, segment_ids
    
    tokens_a = example.text_a
    if len(tokens_a) > max_seq_length-2:
        tokens_a = tokens_a[0 : (max_seq_length-2)]

# Token map will be an int -> int mapping between the `orig_tokens` index and
# the `bert_tokens` index.

# bert_tokens == ["[CLS]", "john", "johan", "##son", "'", "s", "house", "[SEP]"]
# orig_to_tok_map == [1, 2, 4, 6]   
    orig_to_tok_map = []              
    tokens = []
    segment_ids = []
    
    tokens.append("[CLS]")
    segment_ids.append(0)
    orig_to_tok_map.append(len(tokens)-1)
    #print(len(tokens_a))
    for token in tokens_a:
        orig_to_tok_map.append(len(tokens))       
        tokens.extend(tokenizer.tokenize(token))
        #orig_to_tok_map.append(len(tokens)-1)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)
    orig_to_tok_map.append(len(tokens)-1)
    input_ids = tokenizer.convert_tokens_to_ids([tokens[i] for i in orig_to_tok_map])
    #print(len(orig_to_tok_map), len(tokens), len(input_ids), len(segment_ids)) #for debugging

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)
    #print(len(label_ids)) #for debugging
    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    return input_ids, input_mask, segment_ids

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids = [], [], []
    for example in examples:
        input_id, input_mask, segment_id = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        
    )

def convert_text_to_examples(texts):
    """Create InputExamples"""
    InputExamples = []
    for text in texts:
        InputExamples.append(
            InputExample(guid=None, text_a=text, text_b=None)
        )
    return InputExamples

In [20]:
MAX_SEQUENCE_LENGTH = 110

In [21]:
tokenizer = tok.ThaiTokenizer('th.wiki.bpe.op25000.vocab', 'th.wiki.bpe.op25000.model')

In [54]:
# Convert data to InputExample format
examples = convert_text_to_examples(text)

In [55]:
(train_input_ids, train_input_masks, train_segment_ids
) = convert_examples_to_features(tokenizer, examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [30]:
punctuation += 'ฯ'
punctuation += ' '
all_thai_digits = [num for num in thai_digits]+ '1 2 3 4 5 6 7 8 9 0'.split(' ')
thai_num = 'หนึ่ง สอง สาม สี่ ห้า หก เจ็ด แปด เก้า สิบ เอ็ด ยี่ ร้อย พัน หมื่น แสน ล้าน'.split(' ')
english_check = re.compile(r'[a-zA-Z]')
abbr_check = re.compile(r'[^\.]*\.[^\.]\.?')

def is_thai_alpha(word):
    return int(bool(re.match(r'^[ก-๏\s]+$', word)))

def is_roman_alpha(word):
    return int(bool(re.match(r'^[a-zA-Z\s]+$', word)))

def is_alnum(word):
    return int(bool(re.match(r'^[ก-๙a-zA-Z0-9\s]+$', word)))

def is_capitalized(word):
    return int(bool(re.match(r'^[A-Z][a-z]+(\s[A-Z][a-z]+)*$', word)))

def is_punct(word):
    return int(bool(re.match(r'^[{}\s]+$'.format(punctuation), word)))      

def is_num(word):
    return int(bool(re.match(r'^[๐-๙0-9]+$', word)))

def orthog(word):
    return {
        '1is_roman': is_roman_alpha(word),
        '2is_thai': is_thai_alpha(word),
        '3all_digit': is_num(word),
        '4has_digit': int(any([char in word for char in all_thai_digits])),
        '5all_punct': is_punct(word),
        '6has_punct': int(any([char in word for char in punctuation])),
    }

def orthog_to_vector(feature_dict):
    feature_vec = np.zeros(len(feature_dict))
    for i, k in enumerate(sorted(feature_dict.keys())):
        feature_vec[i] = feature_dict[k]
    return feature_vec

def all_orthog_vec(X):
    vec = np.zeros((len(X), MAX_SEQUENCE_LENGTH+2, 6))
    for i, seq in enumerate(X):
        for j, word in enumerate(seq):
            feature = orthog(word)
            for k, feat in enumerate(sorted(feature.keys())):
                vec[i][j+1][k] = feature[feat]
    return vec

In [28]:
train_orthog = all_orthog_vec([text])

In [29]:
#config = BertConfig.from_pretrained('bert_base_th')
bert = TFBertModel.from_pretrained('bert_base_th')

Some weights of the model checkpoint at bert_base_th were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert_base_th.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [34]:
def build_model(max_seq_length):
    seed = 0 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids", dtype='int64')
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks", dtype='int64')
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids", dtype='int64')
    orthog_in = tf.keras.layers.Input(shape=(max_seq_length, 6))
    bert_inputs = [in_id, in_mask, in_segment]
    all_inputs = bert_inputs + [orthog_in]
    bert_output = bert(bert_inputs)[0]
    x = concatenate([bert_output, orthog_in], axis=-1)
    x = tf.keras.layers.SpatialDropout1D(0.3)(x)
    out = tf.keras.layers.Dense(n_tags, activation=tf.keras.activations.softmax)(x)
    model = tf.keras.models.Model(inputs=all_inputs, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00004), loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])   
    model.summary()
    return model

In [24]:
n_tags = 14

In [33]:
model = build_model(MAX_SEQUENCE_LENGTH+2) 
model.load_weights('AACL_BERT_TH.hdf5')

Model: "model"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_ids (InputLayer)           [(None, 112)]         0                                            
____________________________________________________________________________________________________
input_masks (InputLayer)         [(None, 112)]         0                                            
____________________________________________________________________________________________________
segment_ids (InputLayer)         [(None, 112)]         0                                            
____________________________________________________________________________________________________
tf_bert_model (TFBertModel)      ((None, 112, 768), (N 105244416   input_ids[0][0]                  
                                                                   input_mas

In [35]:
int2tag = {0: '-PAD-',
 1: 'ADP',
 2: 'ADV',
 3: 'AUX',
 4: 'CCONJ',
 5: 'DET',
 6: 'NOUN',
 7: 'NUM',
 8: 'PART',
 9: 'PRON',
 10: 'PROPN',
 11: 'PUNCT',
 12: 'SCONJ',
 13: 'VERB'}

In [49]:
def tag(text):
    """
    text = list of tokenized_words
    """
    examples = convert_text_to_examples([text]
                                        )
    input_ids, input_masks, segment_ids = convert_examples_to_features(tokenizer, examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)
    orthog = all_orthog_vec([text])
    predictions = model.predict([input_ids, input_masks, segment_ids, orthog]).argmax(axis=-1)
    predictions =  [[int2tag[i] for i in seq if i!=0] for seq in predictions]
    return tuple(zip(text, predictions[0]))

In [44]:
text = word_tokenize('การตรวจสอบความเป็นส่วนตัว')
print(text)

['การ', 'ตรวจสอบ', 'ความเป็นส่วนตัว']


In [50]:
tag(text)

(('การ', 'NOUN'), ('ตรวจสอบ', 'VERB'), ('ความเป็นส่วนตัว', 'NOUN'))